In [ ]:
!pip install -q keras-nlp --upgrade

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import keras_nlp
import keras
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch
import tensorflow as tf
import os

keras.mixed_precision.set_global_policy("mixed_float16")

label_names = ["entailment", "neutral", "contradiction"]

In [ ]:
DATA_DIR = '/kaggle/input/contradictory-my-dear-watson/'
for dirname, _, filenames in os.walk(DATA_DIR):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df_train = pd.read_csv(DATA_DIR + "train.csv")
df_train.head()

In [ ]:
def split_labels(x, y):
    return (x[0], x[1]), y

In [ ]:
batch_size = 28
buffer_size = batch_size * 10

training_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            df_train[["hypothesis", "premise"]].values,
            df_train["label"].values
        )
    )
)

print(f"Size of the train dataset: {len(training_dataset)}")

train_preprocessed = training_dataset.shuffle(buffer_size=buffer_size).map(split_labels, tf.data.AUTOTUNE).batch(batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

In [ ]:
electra_discriminator_token = keras_nlp.models.Tokenizer.from_preset("electra_small_discriminator_uncased_en")
bert_preprocessor = keras_nlp.models.BertTextClassifierPreprocessor(electra_discriminator_token, sequence_length=240)

train_set = (
    train_preprocessed.map(bert_preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)
)

electra_discriminator_back = keras_nlp.models.Backbone.from_preset("electra_small_discriminator_uncased_en")

electra_classifier = keras_nlp.models.BertClassifier(electra_discriminator_back, 3, preprocessor=None)

classifier_history = electra_classifier.fit(train_set, epochs=5)
training_losses = classifier_history.history["loss"]



### Predictions on test data for submission

In [ ]:
df_test = pd.read_csv(DATA_DIR + "test.csv")
print(df_test.head())
df_test["label"] = [0] * len(df_test)

In [ ]:
testing_dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            df_test[["hypothesis", "premise"]].values,
            df_test["label"].values
        )
    )
)

print(len(testing_dataset))

test_preprocessed = testing_dataset.map(split_labels, tf.data.AUTOTUNE).batch(1, drop_remainder=True).cache().prefetch(tf.data.AUTOTUNE)
bert_test_set = (test_preprocessed.map(bert_preprocessor, tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE))

In [ ]:
predictions = electra_classifier.predict(bert_test_set)

In [ ]:
predicted_classes = tf.argmax(predictions, axis=1)
predicted_classes_np = predicted_classes.numpy()
print(predicted_classes_np)

In [ ]:
submission = df_test.id.copy().to_frame()
submission["prediction"] = predicted_classes_np
submission.to_csv("submission.csv", index=False)

submission